In [9]:
print('Baseline')

# This is so that you don't have to restart the kernel everytime you edit hmm.py

%load_ext autoreload
%autoreload 2

Baseline
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from preprocess import load_arguments, load_label, load_values_from_json, combine_columns, split_arguments
from model import train_bert_model
import os

# Load arguments
df_arguments = load_arguments('data/arguments-training.tsv')

# Load json
human_values = load_values_from_json('data/value-categories.json')

df_labels = load_label("data/labels-training.tsv", human_values["1"])
df_full_level = combine_columns(df_arguments, df_labels)
# train_arguments, valid_arguments = split_arguments(df_full_level)

train_arguments = df_full_level

df_valid_arguments = load_arguments('data/arguments-validation.tsv')
df_valid_labels = load_label("data/labels-validation.tsv", human_values["1"])
df_valid_full_level = combine_columns(df_valid_arguments, df_valid_labels)

valid_arguments = df_valid_full_level

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176 entries, 3401 to 3031
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Argument ID                 4176 non-null   object
 1   Conclusion                  4176 non-null   object
 2   Stance                      4176 non-null   object
 3   Premise                     4176 non-null   object
 4   Achievement                 4176 non-null   int64 
 5   Benevolence: caring         4176 non-null   int64 
 6   Benevolence: dependability  4176 non-null   int64 
 7   Conformity: interpersonal   4176 non-null   int64 
 8   Conformity: rules           4176 non-null   int64 
 9   Face                        4176 non-null   int64 
 10  Hedonism                    4176 non-null   int64 
 11  Humility                    4176 non-null   int64 
 12  Power: dominance            4176 non-null   int64 
 13  Power: resources            4176 non-null   i

## Random Baseline

In [11]:
from pprint import pprint

from baseline import random_prediction
from metrics import compute_metrics, print_metrics
from utils import extract_true_labels



random_prediction_values = random_prediction(human_values['1'], train_arguments)

metrics = compute_metrics((random_prediction_values, 
                           extract_true_labels(train_arguments, human_values['1'])), human_values['1'])

pprint(metrics)

Random Prediction
Added predictions for 4176 arguments
{'accuracy_thresh': 0.27721503376960754,
 'f1-score': {'Achievement': 0.403,
              'Benevolence: caring': 0.401,
              'Benevolence: dependability': 0.254,
              'Conformity: interpersonal': 0.082,
              'Conformity: rules': 0.356,
              'Face': 0.135,
              'Hedonism': 0.078,
              'Humility': 0.157,
              'Power: dominance': 0.16,
              'Power: resources': 0.182,
              'Security: personal': 0.524,
              'Security: societal': 0.447,
              'Self-direction: action': 0.408,
              'Self-direction: thought': 0.297,
              'Stimulation': 0.107,
              'Tradition': 0.197,
              'Universalism: concern': 0.476,
              'Universalism: nature': 0.124,
              'Universalism: objectivity': 0.29,
              'Universalism: tolerance': 0.227,
              'avg-f1-score': 0.265},
 'marco-avg-f1score': 0.265}

## All 1 predictions


In [12]:
from baseline import all_ones

one_values = all_ones(human_values['1'], train_arguments)

metrics = compute_metrics((one_values, 
                           extract_true_labels(train_arguments, human_values['1'])), human_values['1'])

pprint(metrics)

Predicting all 1
Added predictions for 4176 arguments
{'accuracy_thresh': 0.17182710766792297,
 'f1-score': {'Achievement': 0.427,
              'Benevolence: caring': 0.452,
              'Benevolence: dependability': 0.258,
              'Conformity: interpersonal': 0.081,
              'Conformity: rules': 0.376,
              'Face': 0.135,
              'Hedonism': 0.08,
              'Humility': 0.153,
              'Power: dominance': 0.164,
              'Power: resources': 0.193,
              'Security: personal': 0.547,
              'Security: societal': 0.477,
              'Self-direction: action': 0.407,
              'Self-direction: thought': 0.304,
              'Stimulation': 0.114,
              'Tradition': 0.2,
              'Universalism: concern': 0.55,
              'Universalism: nature': 0.129,
              'Universalism: objectivity': 0.301,
              'Universalism: tolerance': 0.242,
              'avg-f1-score': 0.28},
 'marco-avg-f1score': 0.28}


In [ ]:
bert_model_evaluation = train_bert_model(train_arguments, os.path.join('model/', 'prajjwal1/bert-small'), human_values['1'], valid_arguments)
print(bert_model_evaluation['eval_f1-score'])

  0%|          | 0/4176 [00:00<?, ?ex/s]

  0%|          | 0/1044 [00:00<?, ?ex/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at C:\Users\akshi/.cache\huggingface\hub\models--prajjwal1--bert-small\snapshots\0ec5f86f27c1a77d704439db5e01c307ea11b9d4\config.json
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-small",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "L

Step,Training Loss,Validation Loss,Accuracy Thresh,F1-score,Marco-avg-f1score
500,0.430600,0.380950,0.846169,"{'Achievement': 0.27, 'Benevolence: caring': 0.05, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.0, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.0, 'Security: personal': 0.59, 'Security: societal': 0.5, 'Self-direction: action': 0.02, 'Self-direction: thought': 0.0, 'Stimulation': 0.0, 'Tradition': 0.0, 'Universalism: concern': 0.52, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.0, 'Universalism: tolerance': 0.0, 'avg-f1-score': 0.1}",0.100000
1000,0.372600,0.354642,0.857711,"{'Achievement': 0.58, 'Benevolence: caring': 0.28, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.05, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.29, 'Security: personal': 0.68, 'Security: societal': 0.56, 'Self-direction: action': 0.35, 'Self-direction: thought': 0.46, 'Stimulation': 0.0, 'Tradition': 0.04, 'Universalism: concern': 0.51, 'Universalism: nature': 0.27, 'Universalism: objectivity': 0.0, 'Universalism: tolerance': 0.0, 'avg-f1-score': 0.2}",0.200000
1500,0.341400,0.341015,0.863171,"{'Achievement': 0.56, 'Benevolence: caring': 0.37, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.22, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.32, 'Security: personal': 0.68, 'Security: societal': 0.67, 'Self-direction: action': 0.46, 'Self-direction: thought': 0.55, 'Stimulation': 0.0, 'Tradition': 0.15, 'Universalism: concern': 0.66, 'Universalism: nature': 0.7, 'Universalism: objectivity': 0.1, 'Universalism: tolerance': 0.03, 'avg-f1-score': 0.27}",0.270000
2000,0.323300,0.334286,0.866188,"{'Achievement': 0.58, 'Benevolence: caring': 0.42, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.27, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.44, 'Security: personal': 0.7, 'Security: societal': 0.63, 'Self-direction: action': 0.59, 'Self-direction: thought': 0.57, 'Stimulation': 0.0, 'Tradition': 0.3, 'Universalism: concern': 0.62, 'Universalism: nature': 0.72, 'Universalism: objectivity': 0.17, 'Universalism: tolerance': 0.15, 'avg-f1-score': 0.31}",0.310000
2500,0.303000,0.330607,0.866715,"{'Achievement': 0.6, 'Benevolence: caring': 0.44, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.35, 'Face': 0.0, 'Hedonism': 0.13, 'Humility': 0.0, 'Power: dominance': 0.04, 'Power: resources': 0.49, 'Security: personal': 0.69, 'Security: societal': 0.65, 'Self-direction: action': 0.54, 'Self-direction: thought': 0.59, 'Stimulation': 0.03, 'Tradition': 0.34, 'Universalism: concern': 0.69, 'Universalism: nature': 0.74, 'Universalism: objectivity': 0.21, 'Universalism: tolerance': 0.19, 'avg-f1-score': 0.34}",0.340000
3000,0.288100,0.333998,0.864847,"{'Achievement': 0.61, 'Benevolence: caring': 0.45, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.38, 'Face': 0.0, 'Hedonism': 0.18, 'Humility': 0.0, 'Power: dominance': 0.06, 'Power: resources': 0.51, 'Security: personal': 0.67, 'Security: societal': 0.65, 'Self-direction: action': 0.57, 'Self-direction: thought': 0.61, 'Stimulation': 0.1, 'Tradition': 0.41, 'Universalism: concern': 0.69, 'Universalism: nature': 0.77, 'Universalism: objectivity': 0.28, 'Universalism: tolerance': 0.21, 'avg-f1-score': 0.36}",0.360000
3500,0.272500,0.328541,0.867433,"{'Achievement': 0.61, 'Benevolence: caring': 0.44, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.39, 'Face': 0.0, 'Hedonism': 0.13, 'Humility': 0.0, 'Power: dominance': 0.04, 'Power: resources': 0.49, 'Security: personal': 0.68, 'Security: societal': 0.66, 'Self

***** Running Evaluation *****
  Num examples = 1044
  Batch size = 8
Saving model checkpoint to model/prajjwal1/bert-small\checkpoint-500
Configuration saved in model/prajjwal1/bert-small\checkpoint-500\config.json
Model weights saved in model/prajjwal1/bert-small\checkpoint-500\pytorch_model.bin
tokenizer config file saved in model/prajjwal1/bert-small\checkpoint-500\tokenizer_config.json
Special tokens file saved in model/prajjwal1/bert-small\checkpoint-500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1044
  Batch size = 8
Saving model checkpoint to model/prajjwal1/bert-small\checkpoint-1000
Configuration saved in model/prajjwal1/bert-small\checkpoint-1000\config.json
Model weights saved in model/prajjwal1/bert-small\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in model/prajjwal1/bert-small\checkpoint-1000\tokenizer_config.json
Special tokens file saved in model/prajjwal1/bert-small\checkpoint-1000\special_tokens_map.json
***** Running Eva

tokenizer config file saved in model/prajjwal1/bert-small\checkpoint-8500\tokenizer_config.json
Special tokens file saved in model/prajjwal1/bert-small\checkpoint-8500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1044
  Batch size = 8
Saving model checkpoint to model/prajjwal1/bert-small\checkpoint-9000
Configuration saved in model/prajjwal1/bert-small\checkpoint-9000\config.json
Model weights saved in model/prajjwal1/bert-small\checkpoint-9000\pytorch_model.bin
tokenizer config file saved in model/prajjwal1/bert-small\checkpoint-9000\tokenizer_config.json
Special tokens file saved in model/prajjwal1/bert-small\checkpoint-9000\special_tokens_map.json


In [15]:
print(bert_model_evaluation)

{'eval_loss': 0.3503268361091614, 'eval_accuracy_thresh': 0.861398458480835, 'eval_f1-score': {'Achievement': 0.61, 'Benevolence: caring': 0.45, 'Benevolence: dependability': 0.07, 'Conformity: interpersonal': 0.08, 'Conformity: rules': 0.44, 'Face': 0.08, 'Hedonism': 0.22, 'Humility': 0.04, 'Power: dominance': 0.13, 'Power: resources': 0.46, 'Security: personal': 0.67, 'Security: societal': 0.66, 'Self-direction: action': 0.55, 'Self-direction: thought': 0.6, 'Stimulation': 0.22, 'Tradition': 0.45, 'Universalism: concern': 0.68, 'Universalism: nature': 0.76, 'Universalism: objectivity': 0.28, 'Universalism: tolerance': 0.28, 'avg-f1-score': 0.39}, 'eval_marco-avg-f1score': 0.39, 'eval_runtime': 18.0899, 'eval_samples_per_second': 57.712, 'eval_steps_per_second': 7.242, 'epoch': 20.0}
